# Data Preparation


## Robots.txt WARC records of top-10k domains

Objective: extract the robots.txt WARC records of 10,000 top-ranking domains from [Common Crawl's robots.txt dataset](https://commoncrawl.org/2016/09/robotstxt-and-404-redirect-data-sets/).

Note: the general procedure how to extract a records for a large list of domains is described in this [notebook](https://github.com/commoncrawl/cc-notebooks/blob/main/cc-index-table/bulk-url-lookups-by-table-joins.ipynb#Variants-and-Further-Optimizations) in more detail.

- download the domain-level ranks ([cc-main-2022-may-jun-aug-domain-ranks.txt.gz](https://data.commoncrawl.org/projects/hyperlinkgraph/cc-main-2022-may-jun-aug/domain/cc-main-2022-may-jun-aug-domain-ranks.txt.gz)) from [May/Jun/Aug 2022 web graphs](https://commoncrawl.org/2022/09/host-and-domain-level-web-graphs-may-jun-aug-2022/) dataset
- extract the top-10k domains as pair ⟨rank,domain⟩
  ```bash
  zcat data/cc-main-2022-may-jun-aug-domain-ranks.txt.gz \
      | tail -n+2 \
      | head -10000 \
      | cut -f1,5 \
      >data/cc-main-2022-may-jun-aug-domain-ranks-top-10k.txt
  ```
- convert the domain list to [Parquet](https://parquet.apache.org/) file format


In [1]:
import pandas as pd

df = pd.read_csv('data/cc-main-2022-may-jun-aug-domain-ranks-top-10k.txt', sep='\t', names=['rank', 'rev-domain'])
df.head()

,rank,rev-domain
0,1,com.googleapis
1,2,com.facebook
2,3,com.google
3,4,com.twitter
4,5,com.youtube


In [2]:
# unreverse the domain name

def reverse_domain(d):
    parts = d.split('.')
    parts.reverse()
    return '.'.join(parts)

df['domain'] = df['rev-domain'].apply(reverse_domain)
del(df['rev-domain'])
df.head()

,rank,domain
0,1,googleapis.com
1,2,facebook.com
2,3,google.com
3,4,twitter.com
4,5,youtube.com


In [3]:
# save as Parquet
df.to_parquet('data/cc-main-2022-may-jun-aug-domain-ranks-top-10k.parquet.gz', compression='gzip', index=False)

### Bulk lookup by table join

The lookup of all 10k domains in the [columnar index](https://commoncrawl.org/2018/03/index-to-warc-files-and-urls-in-columnar-format/) requires to

- upload the Parquet domain list to S3 (`mybucket` is a placeholder for a bucket in `us-east-1`)
  ```bash
  aws s3 cp data/cc-main-2022-may-jun-aug-domain-ranks-top-10k.parquet.gz s3://mybucket/robotstxt-experiments/domain-top-10k/
  ```
- register the domain list as table in [Amazon Athena](https://aws.amazon.com/athena/)
  - navigate to the [Athena query editor](https://console.aws.amazon.com/athena/home?region=us-east-1#/query-editor) and
  - create a database "robotsexperiments" by executing the following statement:
    ```sql
    CREATE DATABASE robotsexperiments;
    ```
  - register the table "topdomains":
    ```sql
    CREATE EXTERNAL TABLE IF NOT EXISTS robotsexperiments.topdomains (
      `rank`  int,
      `domain` string
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
    WITH SERDEPROPERTIES (
      'serialization.format' = '1'
    ) LOCATION 's3://mybucket/robotstxt-experiments/domain-top-10k/'
    TBLPROPERTIES ('has_encrypted_data'='false');
    ```
  - and verify whether the table is imported properly and contains the expected number of rows
    ```sql
    SELECT * FROM robotsexperiments.topdomains limit 10;
  
    SELECT COUNT(*) FROM robotsexperiments.topdomains;
    ```

Finally, the bulk lookup is done by a table join with the [Common Crawl's columnar index](ttps://commoncrawl.org/2018/03/index-to-warc-files-and-urls-in-columnar-format/), we
- select only the most recent record per same robots.txt URL (the crawler might fetch the robots.txt repeatedly during a monthly crawling running over almost two weeks)
- extract WARC record locations for later processing of robots.txt records
- MIME types (HTTP Content-Type header and identified by content)
- fetch time and status
- and redirect locations (since CC-MAIN-2019-47) in order to "follow" redirects

```sql
WITH allrobots AS (
  SELECT alexa.site,
         alexa.rank,
         cc.url_host_tld,
         cc.url_host_registered_domain,
         cc.url_host_name,
         cc.url,
         cc.fetch_time,
         cc.fetch_status,
         cc.warc_filename,
         cc.warc_record_offset,
         cc.warc_record_length,
         cc.fetch_redirect,
         cc.content_mime_type,
         cc.content_mime_detected,
         -- enumerate records of same URL, most recent first
         ROW_NUMBER() OVER(PARTITION BY cc.url ORDER BY cc.fetch_time DESC) AS n
  FROM "ccindex"."ccindex" AS cc
    RIGHT OUTER JOIN "ccindex"."alexa_top_1m" AS alexa
    ON alexa.domain = cc.url_host_registered_domain
  WHERE cc.crawl = 'CC-MAIN-2022-33'
    AND cc.subset = 'robotstxt'
    -- skip host names which differ from the domain name except for an optional "www." prefix
    AND (length(cc.url_host_name) = length(cc.url_host_registered_domain)
         OR (length(cc.url_host_name) = (length(cc.url_host_registered_domain)+4)
             AND substr(cc.url_host_name, 1, 4) = 'www.')))
SELECT *
 FROM allrobots
-- select only the first (most recent) record of the same URL
WHERE allrobots.n = 1;
```

The query extracts the robots.txt records for a single monthly crawl (CC-MAIN-2022-33). We repeat it for crawls in other years, run in August or September: CC-MAIN-2021-39, CC-MAIN-2020-34, CC-MAIN-2019-35, C-MAIN-2018-34, CC-MAIN-2017-34, C-MAIN-2016-36. Results are stored as `data/cc-main-2022-33-robotstxt-captures.csv` etc.

In [4]:
# read all result files into a dataframe

import glob

robots_captures = glob.glob('data/cc-main-*-robotstxt-captures.csv')

robots_captures_df = []
for f in robots_captures:
    d = pd.read_csv(f, index_col=None)
    d['crawl'] = f[5:20]
    d['year'] = f[13:17]
    robots_captures_df.append(d)

df = pd.concat(robots_captures_df, axis=0, ignore_index=True)

df['year'].value_counts()

2020    164783
2019    137307
2018    123319
2021     99524
2017     78991
2022     72685
2016     21633
Name: year, dtype: int64

In [5]:
df[['content_mime_detected']].value_counts()

content_mime_detected   
text/html                   420944
text/plain                  125032
application/xhtml+xml        47877
application/octet-stream     32970
application/x-msdownload     31096
                             ...  
text/plan                        1
application/zip                  1
audio/vnd.wave                   1
plain/text                       1
video/x-sgi-movie                1
Length: 76, dtype: int64

In [6]:
# extract only "real" robots.txt files (fetch status = HTTP 200, plain text, not HTML)

def is_robotstxt(r: pd.Series):
    if r['fetch_status'] != 200:
        return False
    # plain text MIME types including some noise due to erroneous detections and spell errors
    plain_text_mime_types = {'text/x-robots', 'text/plain', 'message/rfc822', 'text/text', 'text/txt',
                             'text', 'plain/text', 'text/pain', 'text/plan'}
    if (r['content_mime_detected']
        and r['content_mime_detected'] in plain_text_mime_types):
        # since 2018 MIME types are detected based on content
        return True
    if (r['content_mime_type'] in plain_text_mime_types):
        # otherwise we have to rely the HTTP Content-Type header
        return True
    return False

df['is_robotstxt'] = df.apply(is_robotstxt, axis=1)

In [7]:
# over the years we get an almost stable number of "real" robots.txt files

df[['year', 'is_robotstxt']].value_counts().to_frame().sort_values(['year', 'is_robotstxt'])

0
year is_robotstxt        
2016 False           7497
     True           14136
2017 False          66156
     True           12835
2018 False         110929
     True           12390
2019 False         125169
     True           12138
2020 False         152383
     True           12400
2021 False          87381
     True           12143
2022 False          60539
     True           12146

In [8]:
# extract "real" robots.txt records and keep only one record per domain

real_robots_captures = df[df['is_robotstxt']].sort_values(['fetch_time']) \
    .drop_duplicates(['year', 'url_host_registered_domain'], keep='last')

In [9]:
real_robots_captures['year'].value_counts().to_frame().sort_index()

,year
2016,7223
2017,7161
2018,7522
2019,7598
2020,7661
2021,7678
2022,7588


This looks close to a balanced sample:
- we do not have robots.txt files for all 10,000 domains (about 25% missing)
- more files missing for prior years (but that's expected because the domain list is from 2022)

Now, let's dump the lists for fetching the robots.txt captures...

In [10]:
for crawl, d in real_robots_captures.groupby('crawl'):
   d.to_csv('data/{}-robotstxt-captures-selected.csv'.format(crawl), index=False)

### Fetching robots.txt captures

(see [cc-index-table](https://github.com/commoncrawl/cc-index-table#export-subsets-of-the-common-crawl-archives))


### Parsing robots.txt captures

[robotstxt_statistics.py](./robotstxt_statistics.py) based on [cc-pyspark](https://github.com/commoncrawl/cc-pyspark)

